In [24]:
# 導入函式庫
import numpy as np
import tensorflow
from keras.models import Sequential
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils  # 用來後續將 label 標籤轉為 one-hot-encoding  
from matplotlib import pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [25]:
print(tensorflow.__version__)

1.12.0


In [2]:
# # 載入 MNIST 資料庫的訓練資料，並自動分為『訓練組』及『測試組』
# (X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
# print(X_train.shape)

In [4]:
# print(y_train)

In [5]:
# # 建立簡單的線性執行的模型
# model = Sequential()
# # Add Input layer, 隱藏層(hidden layer) 有 256個輸出變數
# model.add(Dense(units=256, input_dim=784, kernel_initializer='normal', activation='relu')) 
# # Add output layer
# model.add(Dense(units=10, kernel_initializer='normal', activation='softmax'))

# # 編譯: 選擇損失函數、優化方法及成效衡量方式
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

# # 將 training 的 label 進行 one-hot encoding，例如數字 7 經過 One-hot encoding 轉換後是 0000001000，即第7個值為 1
# y_TrainOneHot = np_utils.to_categorical(y_train) 
# y_TestOneHot = np_utils.to_categorical(y_test) 

# # 將 training 的 input 資料轉為2維
# X_train_2D = X_train.reshape(60000, 28*28).astype('float32')  
# X_test_2D = X_test.reshape(10000, 28*28).astype('float32')  

# x_Train_norm = X_train_2D/255
# x_Test_norm = X_test_2D/255

# # 進行訓練, 訓練過程會存在 train_history 變數中
# train_history = model.fit(x=x_Train_norm, y=y_TrainOneHot, validation_split=0.2, epochs=10, batch_size=800, verbose=2)  

# # 顯示訓練成果(分數)
# scores = model.evaluate(x_Test_norm, y_TestOneHot)  
# print()  
# print("\t[Info] Accuracy of testing data = {:2.1f}%".format(scores[1]*100.0))  

# # 預測(prediction)
# X = x_Test_norm[0:10,:]
# predictions = model.predict_classes(X)
# # get prediction result
# print(predictions)

In [6]:
model = Sequential()
# 64x64 -> 32x32
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), strides=2, activation='relu', padding='same'))

# 32x32 -> 16x16
#model.add(Conv2D(16, (1, 1), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), strides=2, activation='relu', padding='same'))

# 16x16 -> 8x8
#model.add(Conv2D(32, (1, 1), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), strides=2, activation='relu', padding='same'))

# 8x8 -> 4x4
#model.add(Conv2D(32, (1, 1), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), strides=2, activation='relu', padding='same'))

# 4x4 -> 1x1
model.add(keras.layers.pooling.AveragePooling2D(pool_size=(2,2)))

# flatten and fc
model.add(Flatten())
model.add(Dense(5, activation='softmax'))

#adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
average_pooling2d_1 (Average (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1285      
Total para

---

In [7]:
# train_datagen = ImageDataGenerator(
#         rotation_range=180,
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         validation_split = 0.1)

# test_datagen = ImageDataGenerator(        
#             rotation_range=180,
#             rescale=1./255,
#             shear_range=0.2,
#             zoom_range=0.2,
#             horizontal_flip=True,
#             validation_split = 0.1
# )


data_gen = ImageDataGenerator(   
        rotation_range=180,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split = 0.1)

dir_path = '/Users/huang/Desktop/IDVR_final/TrainTest/train'
batch_size = 32

train_generator = data_gen.flow_from_directory(dir_path,
                                               target_size=(64, 64),
                                               batch_size=batch_size,
                                               class_mode='categorical', subset='training')

validation_generator = data_gen.flow_from_directory(dir_path,
                                               target_size=(64, 64),
                                               batch_size=batch_size,
                                               class_mode='categorical', subset='validation')

# train_generator = train_datagen.flow_from_directory(
#         '/Users/huang/Desktop/IDVR_final/TrainTest/train',
#         target_size=(64, 64),
#         batch_size=32,
#         class_mode='categorical')

# validation_generator = test_datagen.flow_from_directory(
#         '/Users/huang/Desktop/IDVR_final/TrainTest/validation',
#         target_size=(64, 64),
#         batch_size=32,
#         class_mode='categorical')

history = model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        validation_data=validation_generator,
        validation_steps = 10,
#         epochs=23,
        epochs=1
#         validation_data=validation_generator,
#         validation_steps=800
        )

Found 471 images belonging to 5 classes.
Found 51 images belonging to 5 classes.
Epoch 1/1
100/100 [==============================] - 15s 151ms/step - loss: 1.5163 - acc: 0.2386 - val_loss: 1.4970 - val_acc: 0.2549


In [8]:
# model_name = 'model_ver3.h5'
# model.save('/Users/huang/Desktop/IDVR_final/model/' + model_name)

In [22]:
import datetime
import os
from pathlib import Path
count = 1
model_name = str(datetime.date.today())
model_name = ''.join(model_name.split('-')) + '_ver' + str(count)

while( Path('/Users/huang/Desktop/IDVR_final/model/' + model_name+ '.h5').exists() ):
    count += 1
    model_name = model_name[:-1] + str(count)
print(model_name)
model.save('/Users/huang/Desktop/IDVR_final/model/' + model_name + '.h5')

20181227_ver3
